# 本程序用于分析往届《结构力学（2）》的成绩

In [1]:
import pandas as pd
import os

path = './'
files = os.listdir(path)


读入文件生成 DataFrame 的函数

In [2]:
def readFile(fileName):
    return pd.read_csv(fileName, sep='\t')


In [9]:
df = readFile("24251.txt")
df.head()
# 重新生成 index 从 0 开始
df = df.reset_index(drop=True)
df.index

RangeIndex(start=0, stop=87, step=1)